<a href="https://colab.research.google.com/github/Bhuvansai-16/ExcelR-assignemts/blob/main/Chatbot_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#chatbot
import os
import atexit
import shutil
from transformers import BlenderbotTokenizer,BlenderbotForConditionalGeneration
#step1:initialization
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
#step2 defining function
def interact_with_chatbot(user_input,conversation_history):
  #step 2.1: Add user
  conversation_history.append(f"user: {user_input}")
  #2.2 : input text
  input_ids = tokenizer.encode("\n".join(conversation_history)+tokenizer.eos_token,return_tensors='pt')
  #2.3 : Generate a response
  output_ids = model.generate(input_ids,max_length=1000,pad_token_id=tokenizer.eos_token_id)
  response = tokenizer.decode(output_ids[:,input_ids.shape[-1]:][0],skip_special_tokens=True)
  #2.4 : Add the generated response
  conversation_history.append(f"Chatbot: {response}")
  return response,conversation_history

user_input = input("User: ")
conversation_history=[]
interact_with_chatbot(user_input,conversation_history)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

User: hi


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


(" you today? I'm just hanging out with my dog, he's my best friend.",
 ['user: hi',
  "Chatbot:  you today? I'm just hanging out with my dog, he's my best friend."])

In [3]:
import os
import atexit
import shutil
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Step 1: Model Initialization
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Initialize conversation history
conversation_history = []

def interact_with_chatbot(user_input, conversation_history):
    # Step 2.1: Add user input to the conversation history
    conversation_history.append(f"User: {user_input}")

    # Step 2.2: Prepare the input text for the model
    conversation_text = " ".join(conversation_history[-5:])  # Keep only the last 5 exchanges

    # Step 2.3: Generate a response using the chatbot pipeline
    inputs = tokenizer(conversation_text, return_tensors="pt", truncation=True)
    response_ids = model.generate(**inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)

    # Step 2.4: Add the generated response to the conversation history
    conversation_history.append(f"Chatbot: {response_text}")

    return response_text  # Only returning response_text, not conversation_history

# Step 3: Cleaning cache files
def delete_model_file():
    cache_dir = os.path.expanduser("~/.cache/huggingface/hub/models--facebook--blenderbot-400M-distill")
    if os.path.exists(cache_dir):
        user_input = input("Are you sure you want to delete the model files? (yes/no): ")
        if user_input.lower() == "yes":
            shutil.rmtree(cache_dir)
            print(f"Deleted model files from cache directory: {cache_dir}")
        else:
            print("Model files not deleted.")
    else:
        print(f"Model files not found in cache directory: {cache_dir}")

atexit.register(delete_model_file)

# Chatbot Interface
print("Welcome to the Indian Tourism Chatbot!")
print("Type 'quit' to end the conversation.\n")

while True:
    user_input = input("User: ")
    if user_input.lower() == "quit":
        print("Thank you for using the Indian Tourism Chatbot. Have a great day!")
        break
    response = interact_with_chatbot(user_input, conversation_history)  # Fixing unpacking issue
    print(f"Chatbot: {response}")


Welcome to the Indian Tourism Chatbot!
Type 'quit' to end the conversation.

User: quit
Thank you for using the Indian Tourism Chatbot. Have a great day!


In [4]:
import nltk
import re
from nltk.chat.util import Chat, reflections

In [30]:
pairs = [
    [r"my name is (.*)", ["Hello %1, How are you today?"]],
    [r"hi|hey|hello", ["Hello", "Hey there"]],
    [r"what is your name?", ["I am a chatbot created to assist you. You can call me ChatBot."]],
    [r"how are you ?", ["I'm doing good. How about you?"]],
    [r"sorry (.*)", ["Its alright", "Its OK, never mind that"]],
    [r"i'm (.*) doing good", ["Nice to hear that", "Alright, great!"]],
    [r"(.*) age?", ["I'm a computer program, so I don't have an age. But I was created in 2023."]],
    [r"sorry (.*)", ["Its alright", "Its OK, never mind that"]],
    [r"i'm (.*) doing good", ["Nice to hear that", "Alright, great!"]],
    [r"tell me a joke?",["Why are chickens funny?  becoooooze!!"]],
    [r"Best place (.*)",['hyderabad','Smiti valley']]
]

In [31]:
class RBChatbot:
  def __init__(self,pairs):
    self.chat = Chat(pairs,reflections)

  def respond(self,user_input):
    return self.chat.respond(user_input)

chatbot = RBChatbot(pairs)

In [32]:
def chat_with_bot():
  print("Hi, I'm your chatbot Type 'quit' to exit")
  while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
      print("Chatbot: Goodbye!")
      break
    response = chatbot.respond(user_input)
    print("Chatbot:",response)
chat_with_bot()

Hi, I'm your chatbot Type 'quit' to exit
You: Best place in india
Chatbot: hyderabad
You: quit
Chatbot: Goodbye!


In [34]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 776.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [39]:
from transformers import BertForQuestionAnswering,BertTokenizer
import torch
#Load model
model_name = "deepset/bert-base-cased-squad2"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [47]:
context = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in paris,France. It is named after the engineer Gustave Eiffel,Whose company designed and built the tower."""
question = "Who is engineer eiffel tower"

input = tokenizer.encode_plus(question,context,return_tensors='pt')
with torch.no_grad():
  output = model(**input)
  start_scores = output.start_logits
  end_scores = output.end_logits

answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input['input_ids'][0][answer_start:answer_end+1]))
print(f"Answer: {answer}")

Answer: Gustave Eiffel


In [55]:
context = """Coronavirus disease 2019 (COVID-19, also known as SARS-2) is a contagious disease caused
by the coronavirus SARS has spread worldwide,resulting in death of lakhs of people"""
question = "How many people died during COVID-19?"

input = tokenizer.encode_plus(question,context,return_tensors='pt')
with torch.no_grad():
  output = model(**input)
  start_scores = output.start_logits
  end_scores = output.end_logits

answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input['input_ids'][0][answer_start:answer_end+1]))
print(f"Answer: {answer}")

Answer: lakhs
